# Example:

In [ ]:
docs = [
    "This is document 1.",
    "Another example text.",
    "AI-powered systems are fun.",
    "FAISS is a vector search library.",
    "Retrieval-Augmented Generation rocks."
]


In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to index
index.add(embeddings.astype('float32'))

# Save a mapping from FAISS index to original documents
index_to_doc = {i: doc for i, doc in enumerate(docs)}


In [ ]:
query_embedding = model.encode(["What is FAISS?"]).astype('float32')
D, I = index.search(query_embedding, k=3)

print(I)


for idx in I[0]:
    print(index_to_doc[idx])


# For Files:

In [ ]:
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FAISS index
dimension = 384  # MiniLM has 384 dimensions
index = faiss.IndexFlatL2(dimension)

# Metadata mapping
index_to_metadata = {}

# Loop through files
data_dir = "data"
for i, filename in enumerate(os.listdir(data_dir)):
    if filename.endswith(".txt"):
        path = os.path.join(data_dir, filename)
        with open(path, 'r') as f:
            content = f.read()

        # Create embedding
        embedding = model.encode(content).astype('float32')
        index.add(np.array([embedding]))

        # Save title (filename) and content
        index_to_metadata[i] = {"title": filename, "text": content}


In [ ]:
query = "What is FAISS?"
query_embedding = model.encode([query]).astype('float32')

D, I = index.search(query_embedding, k=2)

for idx in I[0]:
    meta = index_to_metadata[idx]
    print(f"Title: {meta['title']}")
    print(f"Content: {meta['text']}\n")
